In [2]:
from datasets import load_dataset
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
from PIL import Image
from sklearn.model_selection import train_test_split
from sklearn.utils import class_weight
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, Input, GlobalAveragePooling2D
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.utils import to_categorical
from autogluon.multimodal import MultiModalPredictor

2024-12-27 12:26:37.322689: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
# Load the preprocessed data
X = np.load("preproc_affectnet/X_affectnet.npy")
y = np.load("preproc_affectnet/y_affectnet.npy")

In [4]:
# Reshape the data (if needed) to add a grayscale channel
if len(X.shape) == 3:
    X = X.reshape(X.shape[0], X.shape[1], X.shape[2], 1)

In [5]:
# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

In [6]:
# One-hot encode the labels
num_classes = len(np.unique(y))  # Determine the number of unique classes
y_train = to_categorical(y_train, num_classes=num_classes)
y_val = to_categorical(y_val, num_classes=num_classes)

In [7]:
# Calculate class weights to handle imbalanced datasets
class_weights = class_weight.compute_class_weight(
    class_weight="balanced",
    classes=np.unique(y),
    y=y
)
class_weights = dict(enumerate(class_weights))

In [8]:
# Define EfficientNetB0 Model
base_model = EfficientNetB0(weights="imagenet", include_top=False, input_shape=(48, 48, 3))
model = Sequential([
    tf.keras.layers.Lambda(lambda x: tf.image.grayscale_to_rgb(x)),  # Convert grayscale to RGB
    base_model,
    GlobalAveragePooling2D(),
    Dense(128, activation="relu"),
    Dropout(0.5),
    Dense(num_classes, activation="softmax")
])

# Compile the model
model.compile(
    optimizer="adam",
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)

In [9]:
# Define callbacks
callbacks = [
    ModelCheckpoint("final_trained_model.keras", monitor="val_accuracy", save_best_only=True, mode="max"),
    EarlyStopping(monitor="val_loss", patience=5, verbose=1),
    ReduceLROnPlateau(monitor="val_loss", factor=0.1, patience=3, verbose=1)
]

In [10]:
# Train the model
history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=20,  # Adjust epochs as needed
    batch_size=32,
    class_weight=class_weights,
    callbacks=callbacks
)

Epoch 1/20
776/776 ━━━━━━━━━━━━━━━━━━━━ 343s 393ms/step - accuracy: 0.3053 - loss: 1.8526 - val_accuracy: 0.1576 - val_loss: 2.2258 - learning_rate: 0.0010
Epoch 2/20
776/776 ━━━━━━━━━━━━━━━━━━━━ 306s 395ms/step - accuracy: 0.5054 - loss: 1.3543 - val_accuracy: 0.2901 - val_loss: 1.8122 - learning_rate: 0.0010
Epoch 3/20
776/776 ━━━━━━━━━━━━━━━━━━━━ 304s 392ms/step - accuracy: 0.5541 - loss: 1.2309 - val_accuracy: 0.1576 - val_loss: 2.9853 - learning_rate: 0.0010
Epoch 4/20
776/776 ━━━━━━━━━━━━━━━━━━━━ 303s 391ms/step - accuracy: 0.6022 - loss: 1.1380 - val_accuracy: 0.1782 - val_loss: 2.5476 - learning_rate: 0.0010
Epoch 5/20
776/776 ━━━━━━━━━━━━━━━━━━━━ 0s 371ms/step - accuracy: 0.6245 - loss: 1.0870  
Epoch 5: ReduceLROnPlateau reducing learning rate to 0.00010000000474974513.
776/776 ━━━━━━━━━━━━━━━━━━━━ 305s 393ms/step - accuracy: 0.6245 - loss: 1.0870 - val_accuracy: 0.1576 - val_loss: 2.5968 - learning_rate: 0.0010
Epoch 6/20
776/776 ━━━━━━━━━━━━━━━━━━━━ 323s 416ms/step - accura

In [11]:
# Evaluate the model on the validation set
val_loss, val_accuracy = model.evaluate(X_val, y_val, verbose=1)
print(f"Validation Accuracy with EfficientNetB0: {val_accuracy * 100:.2f}%")

194/194 ━━━━━━━━━━━━━━━━━━━━ 15s 78ms/step - accuracy: 0.6165 - loss: 1.2496
Validation Accuracy with EfficientNetB0: 62.81%


In [12]:
# AutoGluon MultiModalPredictor
# Prepare the AutoGluon data using the saved dataset
autogluon_data = pd.DataFrame({
    "image": [Image.fromarray((x.squeeze() * 255).astype(np.uint8), "L").convert("RGB") for x in X],
    "label": y
})

In [13]:
# Convert numpy arrays to Pandas DataFrame for AutoGluon compatibility
X_df = pd.DataFrame(X.reshape(X.shape[0], -1))  # Flatten the image data
y_df = pd.DataFrame(y, columns=["label"])  # Add the label column

In [14]:
# Combine features and labels into one DataFrame
autogluon_data = pd.concat([X_df, y_df], axis=1)

In [ ]:
# Initialize and train the AutoGluon predictor
predictor = MultiModalPredictor(label="label")
predictor.fit(autogluon_data, presets="best_quality", time_limit=3600)
print("AutoGluon training completed.")

No path specified. Models will be saved in: "AutogluonModels/ag-20241227_201239"
=================== System Info ===================
AutoGluon Version:  1.2
Python Version:     3.9.21
Operating System:   Darwin
Platform Machine:   x86_64
Platform Version:   Darwin Kernel Version 21.6.0: Mon Jun 24 00:56:10 PDT 2024; root:xnu-8020.240.18.709.2~1/RELEASE_X86_64
CPU Count:          8
Pytorch Version:    2.2.2
CUDA Version:       CUDA is not available
Memory Avail:       6.24 GB / 16.00 GB (39.0%)
Disk Space Avail:   136.15 GB / 233.57 GB (58.3%)
AutoGluon infers your prediction problem is: 'multiclass' (because dtype of label-column == int, but few unique label-values observed).
	8 unique label values:  [0, 1, 2, 3, 4, 5, 6, 7]
	If 'multiclass' is not the correct problem_type, please manually specify the problem_type parameter during Predictor init (You may specify problem_type as one of: ['binary', 'multiclass', 'regression', 'quantile'])

AutoMM starts to create your model. ✨✨✨

To trac

Sanity Checking: |                                        | 0/? [00:00<?, ?it/s]